In [12]:
from numpy import *
import time 
import numpy as np

In [27]:
x = random.random((1800,1800))
v = random.random((18000))

%timeit foo = dot(x, v)


ValueError: shapes (1800,1800) and (18000,) not aligned: 1800 (dim 1) != 18000 (dim 0)

In [14]:
# test sparse matrix multiplication
from scipy import sparse

x1 = sparse.rand(1800,1800)
x1=x1.tobsr()

%timeit foo = x1.dot(v)


10000 loops, best of 3: 33.7 µs per loop


In [15]:
def SpMV_viaMKL( A, x ):
     '''
     Wrapper to Intel's SpMV
     (Sparse Matrix-Vector multiply)
     For medium-sized matrices, this is 4x faster
     than scipy's default implementation
     Stephen Becker, April 24 2014
     stephen.beckr@gmail.com
     '''

     import numpy as np
     import scipy.sparse as sparse
     from ctypes import POINTER,c_void_p,c_int,c_char,c_double,byref,cdll
     mkl = cdll.LoadLibrary("libmkl_rt.so")

     SpMV = mkl.mkl_cspblas_dcsrgemv
     # Dissecting the "cspblas_dcsrgemv" name:
     # "c" - for "c-blas" like interface (as opposed to fortran)
     #    Also means expects sparse arrays to use 0-based indexing, which python does
     # "sp"  for sparse
     # "d"   for double-precision
     # "csr" for compressed row format
     # "ge"  for "general", e.g., the matrix has no special structure such as symmetry
     # "mv"  for "matrix-vector" multiply

     m, n = A.shape

     # The data of the matrix
     data    = A.data.ctypes.data_as(POINTER(c_double))
     indptr  = A.indptr.ctypes.data_as(POINTER(c_int))
     indices = A.indices.ctypes.data_as(POINTER(c_int))

     # Allocate output, using same conventions as input
     nVectors = 1
     if x.ndim is 1:
        y = np.empty(m,dtype=np.double,order='F')
     elif x.shape[1] is 1:
        y = np.empty((m,1),dtype=np.double,order='F')
        

   

     np_x = x.ctypes.data_as(POINTER(c_double))
     np_y = y.ctypes.data_as(POINTER(c_double))
     # now call MKL. This returns the answer in np_y, which links to y
     SpMV(byref(c_char("N")), byref(c_int(m)),data ,indptr, indices, np_x, np_y ) 

     return y

In [16]:
x2  = x1.tocsr()

%timeit foo = SpMV_viaMKL(A=x2,x=v)


The slowest run took 5.73 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 85 µs per loop


In [28]:
import pyculib.sparse as pcu

x4 = sparse.rand(18000,18000)
x4= pcu.bsr_matrix(x4)



In [29]:
GPU = pcu.Sparse(0)

In [30]:
m,n = x4.shape
nnz = x4.nnz
descr = GPU.matdescr()
csrVal = x4.data
csrRowPtr = x4.indptr
csrColInd = x4.indices

y = np.empty(m,dtype=np.double)

In [31]:
%timeit GPU.csrmv('N', m, n, nnz, 1.0, descr, csrVal, csrRowPtr, csrColInd, v, 0.0, y)

100 loops, best of 3: 2.51 ms per loop


In [10]:
y

array([ 19.98789029,  19.87167271,  27.08782767, ...,  23.77493433,
        25.04480498,  22.22962531])

In [32]:
GPU.stream

0

In [33]:
GPU1 = pcu.Sparse(0)

In [34]:
GPU1.stream

0

In [ ]:
GPU.